In [13]:
import pandas as pd
import requests

df = pd.read_csv('CarnegieData/name_skos.csv')
df.insert(2, 'nationalities', pd.Series(dtype=str))

for index in df.index:
        # isolate the data in the 'skos' row
        wikidata_id = df.loc[index, 'skos']
        # isolate the unique wikidata item identifier beginning with 'Q'
        wikidata_id = wikidata_id[wikidata_id.index('Q'):]
        
        # API call to wikidata to get the "countries of citizenship" for this item
        request = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{wikidata_id}/statements?property=P27')
        if len(dict(request.json())) == 0:
            request = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{wikidata_id}/statements?property=P495')
        
        # create blank list to store nationalities
        nationalities = pd.Series(dtype=str)
        
        try:
            # iterate through all countries of citizenship for the composer
            for result in dict(request.json())['P27']:
                #print(result['value']['content'])
                #print(pd.Series([result['value']['content']]))
                nationalities = nationalities.append(pd.Series([result['value']['content']]), ignore_index=True)
        except KeyError:
            try:
                for result in dict(request.json())['P495']:
                    nationalities = nationalities.append(pd.Series([result['value']['content']]), ignore_index=True)
            except KeyError:
                nationalities = nationalities.append(pd.Series(['Q223050']), ignore_index=True)         # stateless
            
        for i in nationalities.index:
            nationalities.loc[i] = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{nationalities[i]}/labels/en').json()
                #print(nationalities.loc[i])
            
        df.at[index, 'nationalities'] = nationalities

In [14]:
df.to_csv('nationalities.csv')

In [6]:
df.head(100)

,composer,skos,nationalities
0,http://data.carnegiehall.org/names/8886,http://www.wikidata.org/entity/Q727702,0 United States of America dtype: object
1,http://data.carnegiehall.org/names/92308,http://www.wikidata.org/entity/Q3093810,0 United States of America dtype: object
2,http://data.carnegiehall.org/names/82882,http://www.wikidata.org/entity/Q202847,0 United States of America dtype: object
3,http://data.carnegiehall.org/names/76687,http://www.wikidata.org/entity/Q483828,0 Iceland dtype: object
4,http://data.carnegiehall.org/names/31775,http://www.wikidata.org/entity/Q234017,0 United States of America dtype: object
...,...,...,...
95,http://data.carnegiehall.org/names/66342,http://www.wikidata.org/entity/Q3018260,0 United States of America dtype: object
96,http://data.carnegiehall.org/names/8574,http://www.wikidata.org/entity/Q743673,0 United States of America dtype: object
97,http://data.carnegiehall.org/names/65049,http://www.wikidata.org/entity/Q43225100,0 United States of America dtype: object
98,http://data.carnegiehall.org/names/15062,http://www.wikidata.org/entity/Q235952,0 United States of America dtype: object


In [15]:
for index in df.index:
    df.at[index, 'nationalities'] = df.at[index, 'nationalities'].to_string()

In [16]:
df.to_csv('CarnegieData/nationalities_new.csv')